In [9]:
# pip install psycopg2
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    host="bwmc2spjnkl3asahsdwf-postgresql.services.clever-cloud.com", 
    database="bwmc2spjnkl3asahsdwf", 
    user="ui9vqx1cbdowhfoap3dq", 
    password="9wNAizIxMMQtBryr1tOZ")

# Creamos el cursor con el objeto conexion
cur = connection.cursor()

horas=[0.5833333,1,6.8,12.6,18.4,24.2,30,26,22,18,14,10,9.25,8.5,7.75,7,12.75,18.5,24.25,30,25.16666667,20.333333,15.5,10.6666667]

In [2]:
# id, osm_id, osm_name, osm_meta, osm_source_id, osm_target_id, clazz, flags, source, target, km, kmh, cost, reverse_cost, x1, y1, x2, y2, something
NONE = 0
ID = 1
STREET_ID = 10
STREET_NAME = 100
SOURCE_NODE_ID = 1000
TARGET_NODE_ID = 10000
DISTANCE = 100000
MAX_SPEED = 1000000
COST = 10000000
REVERSE_COST = 100000000
SOURCE_LONGITUDE = 1000000000
SOURCE_LATITUDE = 10000000000
TARGET_LONGITUDE = 10000000000
TARGET_LATITUDE = 100000000000

def generateQueryByFlags(flags):
  cFlags = str(flags)
  n = len(cFlags)

  flagsForQuery = ''

  index = -1
  while index > -n - 1:
    if cFlags[index] == '1':
      flagVal = pow(10, abs(index) - 1)
      if flagVal == ID:
        flagsForQuery += 'id'
      elif flagVal == STREET_ID:
        flagsForQuery += 'osm_id'
      elif flagVal == STREET_NAME:
        flagsForQuery += 'osm_name'
      elif flagVal == SOURCE_NODE_ID:
        flagsForQuery += 'source'
      elif flagVal == TARGET_NODE_ID:
        flagsForQuery += 'target'
      elif flagVal == DISTANCE:
        flagsForQuery += 'km'
      elif flagVal == MAX_SPEED:
        flagsForQuery += 'kmh'
      elif flagVal == COST:
        flagsForQuery += 'cost'
      elif flagVal == REVERSE_COST:
        flagsForQuery += 'reverse_cost'
      elif flagVal == SOURCE_LONGITUDE:
        flagsForQuery += 'x1'
      elif flagVal == SOURCE_LATITUDE:
        flagsForQuery += 'y1'
      elif flagVal == TARGET_LONGITUDE:
        flagsForQuery += 'x2'
      elif flagVal == TARGET_LATITUDE:
        flagsForQuery += 'y2'
      else:
        continue
      flagsForQuery += '' if index == -n else ','
    index -= 1

  return flagsForQuery

In [3]:
def getIntersectionInformation(source, target, flags=None):
  if not flags or flags == NONE:
    print('nada')
    return []

  flagsForQuery = generateQueryByFlags(flags)
  print(flagsForQuery)

  query=f"SELECT {flagsForQuery} FROM \"hh_2po_4pgr\" WHERE source="+str(source)+" and target="+str(target)
  
  try:
    cur.execute(query)
  except Exception as e:
    connection.commit()
    print(e)
    return []
    
  return cur.fetchall()[0]

intersection = getIntersectionInformation(1, 6, SOURCE_NODE_ID + TARGET_NODE_ID + COST)
print(intersection)

weight = intersection[2]
print(f'weight: {weight}')

source,target,cost
(1, 6, 0.0005568)
weight: 0.0005568


In [4]:
def updateWeightByHour(weight, hour):
  return weight * horas[hour]

newWeight = updateWeightByHour(weight, 0)
print(f'newWeight: {newWeight}')

newWeight: 0.00032479998144000003


In [5]:
def getWeight(source, target):
  query="SELECT * FROM \"hh_2po_4pgr\" WHERE source="+str(source)+" and target="+str(target)
  
  try:
    cur.execute(query)
  except Exception as e:
    connection.commit()
    print(e)
    return
    
  print(cur.fetchall())

getWeight(1, 6)

[(1, 4827820, 'Plaza 2 de Mayo', None, 31035122, 31035142, 15, 1, 1, 6, 0.0389729, 70, 0.0005568, 1000000.0, -77.0427831, -12.0459308, -77.0430896, -12.0460958, '0102000020E61000000400000093D453F5BC4253C01CC2E73D841728C08C5363F8BE4253C0CCD07822881728C0F15A536AC04253C07CBF2C488E1728C00EB3E1FAC14253C046F762DE991728C0')]


In [6]:
def getDistancia(origen, destino):
  query="SELECT * FROM \"hh_2po_4pgr\" WHERE source="+str(origen)+" and target="+str(destino)
  cur.execute(query)
  kms=cur.fetchall()[0][11]
  return kms

In [7]:
def existeRuta(origen, destino):
   query="SELECT * FROM \"hh_2po_4pgr\" WHERE source="+str(origen)+" and target="+str(destino)
   cur.execute(query)
   result=cur.fetchall()
  
   if result == []:
     query="SELECT * FROM \"hh_2po_4pgr\" WHERE source="+str(destino)+" and target="+str(origen)
     cur.execute(query)
     result2=cur.fetchall()
     if result2 == []:
       return False
     else:
       if result2[0][13]== 1000000:
        return False
       else:
          return "Reverse"
   else: 
     return True

print(existeRuta(55943,72))

Reverse


In [8]:
# Cerrar conexión manualmente
connection.close()

In [35]:
class DB_API:
    # constants
    NONE = 0
    ID = 1
    STREET_ID = 10
    STREET_NAME = 100
    SOURCE_ID = 1000
    TARGET_ID = 10000
    DISTANCE = 100000
    MAX_SPEED = 1000000
    COST = 10000000
    REVERSE_COST = 100000000
    SOURCE_LONGITUDE = 1000000000
    SOURCE_LATITUDE = 10000000000
    TARGET_LONGITUDE = 10000000000
    TARGET_LATITUDE = 100000000000

    def __init__(self):
        self.initDbConnection()

    def initDbConnection(self):
        self.connection = psycopg2.connect(
            host="bwmc2spjnkl3asahsdwf-postgresql.services.clever-cloud.com", 
            database="bwmc2spjnkl3asahsdwf", 
            user="ui9vqx1cbdowhfoap3dq", 
            password="9wNAizIxMMQtBryr1tOZ")
        self.cur = self.connection.cursor()

    def endDbConnection(self):
        self.cur.close()
        self.connection.close()

    def __generateQueryByFlags(self, flags):
        cFlags = str(flags)
        n = len(cFlags)

        flagsForQuery = ''

        index = -1
        while index > -n - 1:
            if cFlags[index] == '1':
                flagVal = pow(10, abs(index) - 1)
                if flagVal == self.ID:
                    flagsForQuery += 'id'
                elif flagVal == self.STREET_ID:
                    flagsForQuery += 'osm_id'
                elif flagVal == self.STREET_NAME:
                    flagsForQuery += 'osm_name'
                elif flagVal == self.SOURCE_ID:
                    flagsForQuery += 'source'
                elif flagVal == self.TARGET_ID:
                    flagsForQuery += 'target'
                elif flagVal == self.DISTANCE:
                    flagsForQuery += 'km'
                elif flagVal == self.MAX_SPEED:
                    flagsForQuery += 'kmh'
                elif flagVal == self.COST:
                    flagsForQuery += 'cost'
                elif flagVal == self.REVERSE_COST:
                    flagsForQuery += 'reverse_cost'
                elif flagVal == self.SOURCE_LONGITUDE:
                    flagsForQuery += 'x1'
                elif flagVal == self.SOURCE_LATITUDE:
                    flagsForQuery += 'y1'
                elif flagVal == self.TARGET_LONGITUDE:
                    flagsForQuery += 'x2'
                elif flagVal == self.TARGET_LATITUDE:
                    flagsForQuery += 'y2'
                else:
                    continue
                flagsForQuery += '' if index == -n else ','
            index -= 1

        return flagsForQuery
    
    def getIntersectionInfo(self, source, target, flags=None):
        if not flags or flags == self.NONE:
            print('nada')
            return []

        flagsForQuery = self.__generateQueryByFlags(flags)
        print(flagsForQuery)

        query=f"SELECT {flagsForQuery} FROM \"hh_2po_4pgr\" WHERE source="+str(source)+" and target="+str(target)
  
        try:
            self.cur.execute(query)
        except Exception as e:
            self.connection.commit()
            print(e)
            return []
            
        return self.cur.fetchall()[0]


In [39]:
dbApi = DB_API()

result = dbApi.getIntersectionInfo(1, 6, DB_API.SOURCE_ID + DB_API.TARGET_ID + DB_API.COST)
print(result)

weight = result[2]
print(f'weight: {weight}')

dbApi.endDbConnection()

source,target,cost
(1, 6, 0.0005568)
weight: 0.0005568


In [40]:
def updateWeightByHour(weight, hour):
  return weight * horas[hour]

newWeight = updateWeightByHour(weight, 0)
print(f'newWeight: {newWeight}')

newWeight: 0.00032479998144000003
